In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium

In [2]:
df = pd.read_csv("../data/Geospatial_Coordinates.csv")

In [3]:
df.shape

(103, 3)

In [4]:
import os
api_key = os.environ["FOURSQUARE_API_KEY"]
#YELP_api_key = os.environ["YELP_API_KEY"]

In [5]:
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [113]:
def get_POI(latitude,longitude):
    
    url = "https://api.foursquare.com/v3/places/search?fields=name,location,categories,rating,popularity,price"
    
    headers = {"Accept": "application/json"}
    headers['Authorization'] = api_key
    
    location = [latitude,longitude]
    location_str = [str(x) for x in location]
    location_join = ','.join(location_str)
    
    params_dict = {
                   "query" : "Food",
                   "ll" : location_join,
                   #"categories" : 
                   "radius" : "2000",
                   "limit" : "50"
    }
    
    response = requests.get(url, params_dict, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error {response.status_code}')
    

In [114]:
Restaurant_data = df.apply(
                            lambda row: get_POI(row['Latitude'], row['Longitude']),
                            axis=1)

In [115]:
Restaurant_data[0]['results'][2]['categories']

[{'id': 13199,
  'name': 'Indian Restaurant',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
   'suffix': '.png'}}]

In [116]:
fs_rest_api=[]
def restaurant_POI(data):
    for row in data:
        for val in row['results']:
            try:
                fs_rest_api.append(
                                [
                                   val['name'],
                                   val['categories'][0]['name'],
                                   val['location']['postcode'],
                                   val['popularity'],
                                   val['rating'],
                                   val['price']
                                ]
                                    )
            except KeyError:
                continue
    

In [117]:
restaurant_POI(Restaurant_data)

In [130]:
df_restaurants = pd.DataFrame(fs_rest_api, columns= ['name','category', 'Postal_code','popularity', 'rating','price'])

In [131]:
df_restaurants.head()

,name,category,Postal_code,popularity,rating,price
0,Subway,Deli,M1B 5N6,0.860107,6.4,1
1,Wendy's,Fast Food Restaurant,M1B 6A3,0.995847,6.2,1
2,Harvey's,Fast Food Restaurant,M1B 5N6,0.982336,6.3,2
3,Caribbean Wave,American Restaurant,M1B 5N6,0.988098,6.4,1
4,Swiss Chalet Rotisserie & Grill,Restaurant,M1B 5N6,0.948866,5.2,2


In [132]:
df_restaurants['Postal_code'] = df_restaurants['Postal_code'].apply(lambda x: x[:3])

In [133]:
df_restaurants

,name,category,Postal_code,popularity,rating,price
0,Subway,Deli,M1B,0.860107,6.4,1
1,Wendy's,Fast Food Restaurant,M1B,0.995847,6.2,1
2,Harvey's,Fast Food Restaurant,M1B,0.982336,6.3,2
3,Caribbean Wave,American Restaurant,M1B,0.988098,6.4,1
4,Swiss Chalet Rotisserie & Grill,Restaurant,M1B,0.948866,5.2,2
...,...,...,...,...,...,...
3183,Harvey's,Fast Food Restaurant,M9W,0.985685,6.6,2
3184,Moxie's Classic Grill,American Restaurant,M9W,0.982968,7.6,2
3185,KFC,Fast Food Restaurant,M9W,0.915667,6.2,1
3186,Thai Express,Thai Restaurant,M9W,0.966047,6.4,2
